In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import array

In [ ]:
class Sieve:
    def __init__(self, maxp):
        self._array = array.array("L") # 32-bit unsigned
        self._maxp = ((maxp+63)//64)*64 + 2
        for x in range(self._maxp//64):
            self._array.append(4294967295)
        p = 3
        while p*p <= self._maxp:
            pp = p + p + p
            while pp <= self._maxp:
                self._put(pp)
                pp += p + p
            while True:
                p += 2
                if self._get(p):
                    break
        
    def is_prime(self, p):
        assert(p>1)
        if p == 2:
            return True
        if p%2 == 0:
            return False
        return self._get(p)
        
    def __iter__(self):
        yield 2
        start = 3
        for x in self._array:
            for i in range(32):
                if (x&1)==1:
                    yield start + i + i
                x = x>>1
            start += 64
        
    def _put(self, n):
        # 3->0, 5->1, 7->2 etc.
        nn = (n-3)//2
        dword = nn//32
        bit = nn%32
        mask = 1<<bit
        self._array[dword] &= (4294967295 - mask)
        
    def _get(self, n):
        nn = (n-3)//2
        dword = nn//32
        bit = nn%32
        mask = 1<<bit
        return (self._array[dword] & mask)!=0

Some testing...

In [ ]:
s = Sieve(10)
assert( list(s)[:4] == [2,3,5,7] )

In [ ]:
def is_prime(p):
    assert(p>1)
    x = 2
    while x*x<=p:
        if p%x == 0:
            return False
        x+=1
    return True

assert(not is_prime(9))
assert(is_prime(2))
assert(is_prime(3))
assert(not is_prime(4))
assert( [x for x in range(2,20) if is_prime(x)] == [2,3,5,7,11,13,17,19])

In [ ]:
s = Sieve(1000)
assert(all(is_prime(x) for x in s))

Find $10^6$ primes

In [ ]:
import time
start = time.process_time()
s = Sieve(1000000)
time.process_time() - start

# PIL

In [ ]:
import PIL.Image

In [ ]:
im = PIL.Image.new("1", (1000,1000), 0)
px = im.load()
for num in s:
    x = (num-1)%1000
    y = (num-1)//1000
    px[x,y] = 1
im

# Much larger

In [ ]:
import time
start = time.process_time()
s = Sieve(100000000)
time.process_time() - start

In [ ]:
im = PIL.Image.new("1", (10000,10000), 0)
px = im.load()
for num in s:
    x = (num-1)%10000
    y = (num-1)//10000
    px[x,y] = 1
im

Would be better done in numpy...

In [ ]:
im1 = PIL.Image.new("L", (1000,1000), 0)
px1 = im1.load()
for n in range(1000000):
    x, y = (n%1000), (n//1000)
    summ = 0
    for xx in range(x*10, x*10+10):
        for yy in range(y*10, y*10+10):
            summ += px[xx,yy]
    px1[x,y] = min(255, int((summ / 10) * 255))

In [ ]:
im1

## Prime number theorem

In [ ]:
x = np.asarray([x*100 for x in range(1, 1000001)])
counts = np.zeros_like(x)
for prime in s:
    if prime >= 100000000:
        break
    m = prime // 100
    counts[m] += 1
cumulative_counts = np.cumsum(counts)

In [ ]:
randoms = []
for m in range(1, 1000001):
    upper_count = m * 100 * 1.06 / np.log(m * 100)
    choices = list(range(m*100 - 99, m*100+1))
    # Optionally remove things which "obviously" aren't prime
    choices = [x for x in choices if not(x%2==0 or x%3==0 or x%5==0)]
    choices = np.random.choice(choices, int(upper_count) - len(randoms), replace=False)
    randoms.extend( np.sort(choices) )

In [ ]:
counts = np.zeros_like(x)
for prime in randoms:
    m = prime // 100
    counts[m] += 1
random_cumulative_counts = np.cumsum(counts)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,8))
ax.plot(x, cumulative_counts, label="primes")
ax.plot(x, random_cumulative_counts, label="randoms")
# 1.06 depends on 10^x, as it were
y = 1.06 * x / np.log(x)
ax.plot(x, y, label="expected")
ax.legend()
pass

So numbers of primes $<=x$ is about $1.06 x / log(x)$

In [ ]:
im = PIL.Image.new("1", (10000,10000), 0)
px = im.load()
for num in randoms:
    x = (num-1)%10000
    y = (num-1)//10000
    px[int(x),int(y)] = 1
im

In [ ]:
im2 = PIL.Image.new("L", (1000,1000), 0)
px2 = im2.load()
for n in range(1000000):
    x, y = (n%1000), (n//1000)
    summ = 0
    for xx in range(x*10, x*10+10):
        for yy in range(y*10, y*10+10):
            summ += px[xx,yy]
    px2[x,y] = min(255, int((summ / 10) * 255))

In [ ]:
im2

In [ ]:
im1.save("primes.png")
im2.save("randoms.png")

In [ ]:
im3 = PIL.Image.new("L", (1000,1000), 0)
im3.paste(im1, (0,0))
im3.paste(im2, (500,0))
im3

In [ ]:
im3.save("both.png")